<a href="https://colab.research.google.com/github/GruheshKurra/FMML_Project_and_Labs/blob/main/AIML_III_Module_01_Lab_02_Machine_Learning_terms_and_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


## Try it out for yourself and answer:
### 1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?

**Answer** : When we increase the percentage of the validation set, we have more data to check how well our model is performing, but less data to train the model. For example, if we go from 20% to 50% validation data, our training data drops from 80% to 50%. This can make the model less accurate because it has less data to learn from. The model might not capture all the patterns in the data, leading to lower accuracy on the validation set.

On the other hand, if we reduce the validation set percentage, we have more training data. For example, going from 20% to 10% validation data means our training data increases from 80% to 90%. This can make the model learn better and perform better on the validation set because it has more data to understand the patterns. However, too small a validation set might not give us a reliable measure of accuracy since it represents a smaller portion of the data.

---

### 2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

**Answer** : The size of the training and validation sets affects our ability to predict test set accuracy.

*   **Small Training Set**: If the training set is too small, like 50% or less, the model doesn’t learn enough patterns. For instance, if we have only 10% of the data for training, the model might perform poorly because it hasn’t seen enough examples.
*   **Small Validation Set**: If the validation set is too small, say 5% or less, the accuracy we get from it might not be reliable. For example, using just 5% of the data for validation might show higher or lower accuracy by chance, not because the model is good or bad.

---

### 3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

**Answer** : A good percentage to reserve for the validation set is around 20% to 30%. This means if we have 1000 data points, we use 200-300 for validation and 700-800 for training.

*   **Enough Training Data**: With 70%-80% data for training, the model can learn well and understand the patterns in the data. For example, training with 700 data points instead of 500 makes the model better.

*   **Reliable Validation**: Using 20%-30% data for validation, like 200-300 points out of 1000, gives us a good measure of how the model will perform on new, unseen data.

---

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Training and validating the 1-nearest neighbor classifier
knn_1 = KNeighborsClassifier(n_neighbors=1)
knn_1.fit(traindata, trainlabel)
val_pred_1 = knn_1.predict(valdata)
val_accuracy_1 = accuracy_score(vallabel, val_pred_1)
print(f'Validation accuracy using 1-nearest neighbor classifier: {val_accuracy_1*100:.2f}%')

# Training and validating the 3-nearest neighbor classifier
knn_3 = KNeighborsClassifier(n_neighbors=3)
knn_3.fit(traindata, trainlabel)
val_pred_3 = knn_3.predict(valdata)
val_accuracy_3 = accuracy_score(vallabel, val_pred_3)
print(f'Validation accuracy using 3-nearest neighbor classifier: {val_accuracy_3*100:.2f}%')

# Testing the 1-nearest neighbor classifier on the test set
test_pred_1 = knn_1.predict(testdata)
test_accuracy_1 = accuracy_score(testlabel, test_pred_1)
print(f'Test accuracy using 1-nearest neighbor classifier: {test_accuracy_1*100:.2f}%')

# Testing the 3-nearest neighbor classifier on the test set
test_pred_3 = knn_3.predict(testdata)
test_accuracy_3 = accuracy_score(testlabel, test_pred_3)
print(f'Test accuracy using 3-nearest neighbor classifier: {test_accuracy_3*100:.2f}%')


Validation accuracy using 1-nearest neighbor classifier: 34.05%
Validation accuracy using 3-nearest neighbor classifier: 34.07%
Test accuracy using 1-nearest neighbor classifier: 34.05%
Test accuracy using 3-nearest neighbor classifier: 34.60%


**Results**:
Validation accuracy for the 1-nearest neighbor (1NN) classifier was 34.05%, while the 3-nearest neighbor (3NN) classifier achieved a slightly higher validation accuracy of 34.07%. On the test dataset, the 1NN classifier had an accuracy of 34.05%, whereas the 3NN classifier improved to 34.60%.

**Conclusion**:
The 3-nearest neighbor (3NN) classifier performed slightly better than the 1-nearest neighbor (1NN) classifier. The validation accuracy was very close for both models, with 1NN at 34.05% and 3NN at 34.07%. On the test set, 1NN achieved 34.05%, while 3NN improved to 34.60%.

These results suggest that the 3NN classifier generalizes better to new data, likely due to its consideration of multiple neighbors, which reduces the impact of noise and outliers. Therefore, using the 3NN classifier is advantageous, especially in noisy datasets. Further experimentation with different
k values and cross-validation can help confirm these findings.

In conclusion, the 3NN classifier is slightly better for this dataset, showing improved test accuracy and robustness.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

## Questions
### 1. Does averaging the validation accuracy across multiple splits give more consistent results?

**Answer**:
Yes, averaging the validation accuracy across multiple splits generally gives more consistent results. This approach reduces the impact of any single split that might have had particularly good or bad performance due to random chance. By averaging over multiple splits, the variance in the accuracy estimates is reduced, leading to a more stable and reliable measure of the model’s performance.

### 2. Does it give more accurate estimate of test accuracy?

**Answer**:
Yes, averaging validation accuracy across multiple splits tends to provide a more accurate estimate of test accuracy. Since it takes into account multiple subsets of data, it better represents the performance of the model on unseen data. This method helps mitigate the bias that could be introduced by any single split and gives a more realistic picture of how the model is likely to perform on the test set.

### 3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

**Answer**:
Increasing the number of iterations typically improves the accuracy estimate. With more iterations, the model’s performance is averaged over a greater number of splits, which helps smooth out anomalies and provides a more robust measure of the model’s true performance. Essentially, higher iterations lead to a better and more reliable estimate of accuracy, as the effect of any single outlier split diminishes.

### 4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

**Answer**:
Increasing the number of iterations can help when dealing with a very small train or validation dataset, but it has its limits. More iterations can provide a better estimate by leveraging different parts of the data multiple times, which helps in understanding the model’s performance more comprehensively. However, if the dataset is too small, the model might still struggle to learn effectively due to insufficient training data. Thus, while increasing iterations is beneficial, ensuring a minimum threshold of data is also crucial for the model to perform adequately.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Function to implement k-nearest neighbors classifier
def NN_k_neighbors(traindata, trainlabel, testdata, k):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(traindata, trainlabel)
    return knn.predict(testdata)

# Function to calculate average accuracy
def AverageAccuracy_k(alldata, alllabel, splitpercent, iterations, k):
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
        valpred = NN_k_neighbors(traindata, trainlabel, valdata, k)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

# Parameters for the experiment
split_sizes = [0.6, 0.7, 0.8]  # Different split sizes (60%, 70%, 80% training data)
num_splits = [5, 10, 15]  # Different numbers of splits (iterations)
k_values = [1, 3]  # k values for 1NN and 3NN

# Experiment
results = {}

for k in k_values:
    for split_size in split_sizes:
        for num_split in num_splits:
            avg_acc = AverageAccuracy_k(alltraindata, alltrainlabel, split_size, num_split, k)
            results[(k, split_size, num_split)] = avg_acc
            print(f'k={k}, split_size={split_size}, num_splits={num_split}: Average validation accuracy = {avg_acc*100:.2f}%')

# Summarize results
print("\nSummary of results:")
for key, value in results.items():
    print(f'k={key[0]}, split_size={key[1]}, num_splits={key[2]}: Average validation accuracy = {value*100:.2f}%')

k=1, split_size=0.6, num_splits=5: Average validation accuracy = 33.72%
k=1, split_size=0.6, num_splits=10: Average validation accuracy = 33.74%
k=1, split_size=0.6, num_splits=15: Average validation accuracy = 33.31%
k=1, split_size=0.7, num_splits=5: Average validation accuracy = 33.93%
k=1, split_size=0.7, num_splits=10: Average validation accuracy = 33.82%
k=1, split_size=0.7, num_splits=15: Average validation accuracy = 34.19%
k=1, split_size=0.8, num_splits=5: Average validation accuracy = 34.03%
k=1, split_size=0.8, num_splits=10: Average validation accuracy = 34.05%
k=1, split_size=0.8, num_splits=15: Average validation accuracy = 34.12%
k=3, split_size=0.6, num_splits=5: Average validation accuracy = 33.62%
k=3, split_size=0.6, num_splits=10: Average validation accuracy = 33.67%
k=3, split_size=0.6, num_splits=15: Average validation accuracy = 33.97%
k=3, split_size=0.7, num_splits=5: Average validation accuracy = 34.43%
k=3, split_size=0.7, num_splits=10: Average validation a

### Conclusion:

**Exercise:** Implement a 3-nearest neighbor classifier and compare the accuracy of the 1-nearest neighbor classifier and the 3-nearest neighbor classifier on the test dataset. Investigate how the accuracy changes with the number of splits and how it is affected by the split size.

**Results:**

- **Validation Accuracy for 1-Nearest Neighbor (1NN):**
  - **Split Size 0.6:**
    - 5 splits: 33.72%
    - 10 splits: 33.74%
    - 15 splits: 33.31%
  - **Split Size 0.7:**
    - 5 splits: 33.93%
    - 10 splits: 33.82%
    - 15 splits: 34.19%
  - **Split Size 0.8:**
    - 5 splits: 34.03%
    - 10 splits: 34.05%
    - 15 splits: 34.12%

- **Validation Accuracy for 3-Nearest Neighbor (3NN):**
  - **Split Size 0.6:**
    - 5 splits: 33.62%
    - 10 splits: 33.67%
    - 15 splits: 33.97%
  - **Split Size 0.7:**
    - 5 splits: 34.43%
    - 10 splits: 34.38%
    - 15 splits: 34.68%
  - **Split Size 0.8:**
    - 5 splits: 34.63%
    - 10 splits: 34.89%
    - 15 splits: 34.55%

### Explanation:

The accuracy of the 3-nearest neighbor (3NN) classifier changes with the number of splits. Specifically, the accuracy of the 3NN classifier increases slightly with the number of splits. For instance, with a split size of 0.6, the accuracy improves from 33.62% with 5 splits to 33.97% with 15 splits. This indicates that more splits generally lead to a more consistent and reliable measure of accuracy, as the effect of random variations in the splits is reduced.

Moreover, the accuracy is also affected by the split size. Increasing the split size (i.e., using more data for training) generally improves the accuracy for both 1NN and 3NN classifiers. For example, for 3NN with 10 splits, the accuracy increases from 33.67% with a split size of 0.6 to 34.89% with a split size of 0.8. This suggests that having more training data helps the model learn better and perform more accurately.

In comparison between 1NN and 3NN, the 3NN classifier generally performs slightly better than the 1NN classifier. For instance, with a split size of 0.8 and 10 splits, the 3NN achieves an accuracy of 34.89%, compared to 34.05% for the 1NN. This indicates that considering more neighbors helps in making more robust predictions by reducing the impact of outliers and noise.


### Conclusion:

Based on the results obtained, we can conclude that:

- Averaging the validation accuracy across multiple splits does provide more consistent results. This is evident from the slight improvements in accuracy as the number of splits increases.
- A larger split size (more training data) generally leads to better accuracy. This is seen with both the 1NN and 3NN classifiers, where the accuracy improves as the training set size increases.
- The 3NN classifier performs slightly better than the 1NN classifier, suggesting it may be more robust in handling noise and outliers.

In summary, increasing the number of splits and the split size positively impacts the accuracy of both 1NN and 3NN classifiers, with 3NN showing a slight advantage overall. This experiment helps understand the impact of these parameters on the performance of k-nearest neighbor classifiers.